In [156]:
#imports
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Problem 1

In [157]:
img = Image.open('./Lenna.png')

# Convert the image to grayscale using 'L' mode
grayscale_img = img.convert('L')

In [158]:
"""Downsample the image by taking every other pixel in both x and y directions."""
img_array = np.array(grayscale_img)

# Downsample by taking every other pixel in both dimensions
downsampled_img_array = img_array[::2, ::2]

# Downsample for the second time
downsampled_twice_img_array = downsampled_img_array[::2, ::2]

# Convert back to PIL Image
downsampled_img = Image.fromarray(downsampled_img_array)
downsampled_twice_img = Image.fromarray(downsampled_twice_img_array)

In [ ]:
"""Comparison 1"""
plt.figure(figsize=(15, 5))

# Display original grayscale image
plt.subplot(1, 3, 1)
plt.title("Original Grayscale Image")
plt.imshow(grayscale_img, cmap='gray')
plt.axis('off')

# Display downsampled image (scaled to original size for comparison)
plt.subplot(1, 3, 2)
plt.title("Downsampled Image (Original Size)")
plt.imshow(downsampled_img, cmap='gray')
plt.axis('off')

# Display downsampled twice image (scaled to original size for comparison)
plt.subplot(1, 3, 3)
plt.title("Downsampled Image Twice (Original Size)")
plt.imshow(downsampled_twice_img, cmap='gray')
plt.axis('off')

plt.show()

In [160]:
def upsample_image(downsampled_img, times):
    """Upsample the image by inserting empty pixels between each pixel."""
    img_array = np.array(downsampled_img)
    
    for _ in range(times):
        # Get current dimensions
        h, w = img_array.shape[0], img_array.shape[1]
        
        # Create a new array with twice the size, filled with zeros (empty pixels)
        upsampled_img_array = np.zeros((h * 2, w * 2), dtype=img_array.dtype)
        
        # Copy original pixels into the upsampled array at every other index
        upsampled_img_array[::2, ::2] = img_array
        
        # Update img_array for next iteration if upsampled multiple times
        img_array = upsampled_img_array
    
    # Convert back to PIL Image
    upsampled_img = Image.fromarray(upsampled_img_array)
    
    return upsampled_img

In [ ]:
"""Comparison 2"""
plt.figure(figsize=(15, 5))

# Display upsampled image
plt.subplot(1, 3, 1)
plt.title("Upsampled Image")
plt.imshow(upsample_image(downsampled_twice_img, 1), cmap='gray')
plt.axis('off')

# Display upsampled twice image
plt.subplot(1, 3, 2)
plt.title("Upsampled Image Twice")
plt.imshow(upsample_image(downsampled_twice_img, 2), cmap='gray')
plt.axis('off')

plt.show()

# Problem 2

In [162]:
def create_gaussian_kernel(k, s):
    """
    Create a 2D Gaussian kernel.
    """
    ax = np.arange(-k // 2 + 1., k // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * s**2))
    return kernel / np.sum(kernel)

In [163]:
def apply_convolution(image, kernel):
    """
    Apply convolution to the image using the given kernel.
    """
    m, n = kernel.shape
    y, x = image.shape
    y = y - m + 1
    x = x - n + 1
    output = np.zeros((y,x))
    for i in range(y):
        for j in range(x):
            output[i,j] = np.sum(image[i:i+m, j:j+n]*kernel)
    return output

In [164]:
def myGaussianSmoothing(I, k, s):
    """
    Apply Gaussian smoothing to an image I with kernel size k and scaling parameter s (sigma).
    """
    # Ensure kernel size is odd
    if k % 2 == 0:
        raise ValueError("Kernel size should be an odd number")
    
    # Create Gaussian kernel
    kernel = create_gaussian_kernel(k, s)
    
    # Apply convolution
    I_smooth = apply_convolution(I, kernel)
    
    return I_smooth

In [165]:
def plot_images(original, smoothed_images, titles):
    """ Utility function to plot the original and smoothed images. """
    plt.figure(figsize=(12, 6))
    
    # Plot original image
    plt.subplot(2, len(smoothed_images)//2 + 1, 1)
    plt.imshow(original, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    # Plot smoothed images
    for i, (img, title) in enumerate(zip(smoothed_images, titles)):
        plt.subplot(2, len(smoothed_images)//2 + 1, i + 2)
        plt.imshow(img, cmap='gray')
        plt.title(title)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
I = cv2.imread('./Lenna.png', cv2.IMREAD_GRAYSCALE)  # Load a grayscale image

# Part 1: Varying kernel size with fixed sigma
kernel_sizes = [3, 5, 7, 11, 51]
sigma = 1
smoothed_images_k = [myGaussianSmoothing(I, k, sigma) for k in kernel_sizes]
titles_k = [f"k={k}, sigma={sigma}" for k in kernel_sizes]

# Part 2: Varying sigma with fixed kernel size
kernel_size = 11
sigmas = [0.1, 1, 2, 3, 5]
smoothed_images_s = [myGaussianSmoothing(I, kernel_size, s) for s in sigmas]
titles_s = [f"k={kernel_size}, sigma={s}" for s in sigmas]

# Plot results for varying kernel sizes
plot_images(I, smoothed_images_k, titles_k)

# Plot results for varying sigma
plot_images(I, smoothed_images_s, titles_s)

# Problem 3

In [167]:
def median_filter(image, kernel_size):
    """Apply median filter to the image."""
    padded = np.pad(image, kernel_size // 2, mode='edge')
    result = np.zeros_like(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            window = padded[i:i+kernel_size, j:j+kernel_size]
            result[i, j] = np.median(window)
    return result

In [168]:
def process_image_gaussian(downsampled_img, upsample_times, gaussian_k, gaussian_sigma):
    """
    Process the image: upsample, apply Gaussian smoothing, upsample again, apply Gaussian smoothing again.
    """
    # First upsample
    upsampled1 = upsample_image(downsampled_img, upsample_times)
    # Convert to numpy array if it's not already
    upsampled_array1 = np.array(upsampled1)
    
    # First Gaussian smoothing
    smoothed1 = myGaussianSmoothing(upsampled_array1, gaussian_k, gaussian_sigma)
    
    # Second upsample
    upsampled2 = upsample_image(upsampled_array1, upsample_times)
    # Convert to numpy array if it's not already
    upsampled_array2 = np.array(upsampled2)
    
    # Second Gaussian smoothing
    smoothed2 = myGaussianSmoothing(upsampled_array2, gaussian_k, gaussian_sigma)
    
    return upsampled_array1, smoothed1, upsampled_array2, smoothed2

In [169]:
def process_image_median(downsampled_img, upsample_times, median_k):
    """
    Process the image: upsample, apply median filter, upsample again, apply median filter again.
    """
    # First upsample
    upsampled1 = upsample_image(downsampled_img, upsample_times)
    # Convert to numpy array if it's not already
    upsampled_array1 = np.array(upsampled1)
    
    # First median filter
    median_filtered1 = median_filter(upsampled_array1, median_k)
    
    # Second upsample
    upsampled2 = upsample_image(upsampled_array1, upsample_times)
    # Convert to numpy array if it's not already
    upsampled_array2 = np.array(upsampled2)
    
    # Second median filter
    median_filtered2 = median_filter(upsampled_array2, median_k)
    
    return upsampled_array1, median_filtered1, upsampled_array2, median_filtered2

In [170]:
def plot_results_gaussian(original, upsampled1, smoothed1, upsampled2, smoothed2):
    """
    Plot the original, first upsampled, first smoothed, second upsampled, and second smoothed images.
    """
    plt.figure(figsize=(25, 5))
    
    plt.subplot(1, 5, 1)
    plt.imshow(original, cmap='gray')
    plt.title("Original Downsampled")
    plt.axis('off')
    
    plt.subplot(1, 5, 2)
    plt.imshow(upsampled1, cmap='gray')
    plt.title("First Upsampled")
    plt.axis('off')
    
    plt.subplot(1, 5, 3)
    plt.imshow(smoothed1, cmap='gray')
    plt.title("First Gaussian Smoothed")
    plt.axis('off')
    
    plt.subplot(1, 5, 4)
    plt.imshow(upsampled2, cmap='gray')
    plt.title("Second Upsampled")
    plt.axis('off')
    
    plt.subplot(1, 5, 5)
    plt.imshow(smoothed2, cmap='gray')
    plt.title("Second Gaussian Smoothed")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [171]:
def plot_results_median_multi(original, results):
    """
    Plot the original and results for multiple median kernel sizes.
    """
    fig, axes = plt.subplots(4, 5, figsize=(25, 20))
    kernel_sizes = [1, 2, 4, 8]
    
    for row, kernel_size in enumerate(kernel_sizes):
        # Plot original
        axes[row, 0].imshow(original, cmap='gray')
        axes[row, 0].set_title("Original Downsampled")
        axes[row, 0].axis('off')
        
        # Plot results for current kernel size
        upsampled1, median_filtered1, upsampled2, median_filtered2 = results[kernel_size]
        
        axes[row, 1].imshow(upsampled1, cmap='gray')
        axes[row, 1].set_title(f"First Upsampled (k={kernel_size})")
        axes[row, 1].axis('off')
        
        axes[row, 2].imshow(median_filtered1, cmap='gray')
        axes[row, 2].set_title(f"First Median Filtered (k={kernel_size})")
        axes[row, 2].axis('off')
        
        axes[row, 3].imshow(upsampled2, cmap='gray')
        axes[row, 3].set_title(f"Second Upsampled (k={kernel_size})")
        axes[row, 3].axis('off')
        
        axes[row, 4].imshow(median_filtered2, cmap='gray')
        axes[row, 4].set_title(f"Second Median Filtered (k={kernel_size})")
        axes[row, 4].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Process the image with Gaussian smoothing
upsampled1_g, smoothed1, upsampled2_g, smoothed2 = process_image_gaussian(
    downsampled_twice_img, 
    upsample_times=2, 
    gaussian_k=11, 
    gaussian_sigma=1
)

# Plot Gaussian results
plot_results_gaussian(downsampled_twice_img, upsampled1_g, smoothed1, upsampled2_g, smoothed2)

In [ ]:
kernel_sizes = [1, 2, 4, 8]
median_results = {}

for k in kernel_sizes:
    median_results[k] = process_image_median(
        downsampled_twice_img_array, 
        upsample_times=1, 
        median_k=k
    )

# Plot median filtering results
plot_results_median_multi(downsampled_twice_img_array, median_results)

# Problem 4

In [174]:
def add_gaussian_noise(image, mean=0, std=0.1):
    """
    Add Gaussian noise to an image.
    """
    noise = np.random.normal(mean, std, image.shape)
    noisy_image = image + noise
    return np.clip(noisy_image, 0, 1)  # Ensure values are in [0, 1] range

In [175]:
def add_thresholded_noise(image, mean=0, std=0.1, threshold=0.2):
    """
    Add thresholded Gaussian noise to an image.
    """
    noise = np.random.normal(mean, std, image.shape)
    thresholded_noise = np.where(noise > threshold, 1, 0)
    noisy_image = image + thresholded_noise
    return np.clip(noisy_image, 0, 1)  # Ensure values are in [0, 1] range

In [176]:
def plot_images(images, titles):
    """
    Plot multiple images side by side.
    """
    fig, axes = plt.subplots(1, len(images), figsize=(20, 5))
    for ax, image, title in zip(axes, images, titles):
        ax.imshow(image, cmap='gray')
        ax.set_title(title)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Load the original image
original_image = np.array(Image.open('./Lenna.png').convert('L')) / 255.0  # Normalize to [0, 1]

# Add Gaussian noise
noisy_image = add_gaussian_noise(original_image)

# Apply Gaussian smoothing (you can adjust these parameters)
gaussian_smoothed = cv2.GaussianBlur(noisy_image, (5, 5), 1)

# Apply median filtering
median_filtered = median_filter(noisy_image, 3)

# Plot results
plot_images([original_image, noisy_image, gaussian_smoothed, median_filtered],
            ['Original', 'Noisy', 'Gaussian Smoothed', 'Median Filtered'])

In [ ]:
# Now let's add thresholded noise
thresholded_noisy_image = add_thresholded_noise(original_image)

# Apply Gaussian smoothing to thresholded noisy image
gaussian_smoothed_thresholded = cv2.GaussianBlur(thresholded_noisy_image, (5, 5), 1)

# Apply median filtering to thresholded noisy image
median_filtered_thresholded = median_filter(thresholded_noisy_image, 3)

# Plot results for thresholded noise
plot_images([original_image, thresholded_noisy_image, gaussian_smoothed_thresholded, median_filtered_thresholded],
            ['Original', 'Thresholded Noisy', 'Gaussian Smoothed (Thresholded)', 'Median Filtered (Thresholded)'])

# Problem 5

In [179]:
def mySobelFilter(I):
    # Define Sobel kernels
    kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    kernel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    # Apply convolution manually
    grad_x = apply_convolution(I, kernel_x)
    grad_y = apply_convolution(I, kernel_y)

    # Calculate magnitude and orientation
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    orientation = np.arctan2(grad_y, grad_x)

    # Normalize magnitude to [0, 1]
    magnitude = (magnitude - magnitude.min()) / (magnitude.max() - magnitude.min())

    return magnitude, orientation

In [180]:
def visualize_sobel(magnitude, orientation):
    """
    Visualize Sobel filter results using HSV color space.
    """
    # Convert orientation to hue (0-179 for visualization)
    hue = (orientation + np.pi) / (2 * np.pi) * 179  # Normalize to range [0, 179]

    # Use the magnitude for both saturation and value
    saturation = magnitude
    value = magnitude

    # Create HSV image
    hsv = np.zeros((*magnitude.shape, 3), dtype=np.float32)
    hsv[..., 0] = hue / 179.0  # Normalize hue to [0, 1]
    hsv[..., 1] = saturation  # Saturation
    hsv[..., 2] = value  # Value

    # Convert HSV to RGB
    rgb = cv2.cvtColor((hsv*255).astype(np.uint8), cv2.COLOR_HSV2RGB)
    return rgb, hsv

In [ ]:
# Load and preprocess the image
image = np.array(Image.open('./Lenna.png').convert('L'))
image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

# Apply Sobel filter
magnitude, orientation = mySobelFilter(image)

# Visualize results
rgb_visualization, hsv_visualization = visualize_sobel(magnitude, orientation)

# Plot results
plt.figure(figsize=(25, 5))

plt.subplot(1, 5, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 5, 2)
plt.imshow(magnitude)
plt.title('Gradient Magnitude')
plt.axis('off')

plt.subplot(1, 5, 3)
plt.imshow(orientation, cmap='hsv')
plt.title('Gradient Orientation')
plt.axis('off')

plt.subplot(1, 5, 4)
plt.imshow(rgb_visualization)
plt.title('Color Visualization')
plt.axis('off')

plt.subplot(1, 5, 5)
plt.imshow(hsv_visualization)
plt.title('HSV Visualization')
plt.axis('off')

plt.tight_layout()
plt.show()